In [ ]:
from autogen import ConversableAgent, register_function
from tavily import TavilyClient


In [ ]:
# Define the search function using Tavily
def search_internet(query: str) -> str:
    """
    Search the web using Tavily API

    Args:
        query (str): The search query

    Returns:
        str: Search results from Tavily
    """
    client = TavilyClient(api_key='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')
    try:
        response = client.search(query=query)
        return str(response)
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
# Set up the LLM configuration
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": '',
        # "api_type": "google"
    }
]


In [ ]:
# Create the user proxy agent
user_proxy_agent = ConversableAgent(
    name="UserProxy",
    llm_config=False,  # No LLM for this agent
    human_input_mode="NEVER",
    code_execution_config=False,  # No code execution needed
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'."
)

NameError: name 'autogen' is not defined

In [ ]:
# Create the researcher agent
researcher_agent = ConversableAgent(
    name="Researcher",
    llm_config={"config_list": config_list},
    system_message="You are a professional online researcher. Use the search_internet function to find information on the web. Your task is to research the latest GPT-4.5 and provide key facts, figures, and credible sources. Once complete, include the findings in your message and end with 'TERMINATE'."
)

In [ ]:
# Create the writer agent
writer_agent = ConversableAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="You are a professional content writer. Your task is to take research findings and write a well-structured, engaging article with an introduction, clear sections, and proper citations. Once complete, include the article in your message and end with 'TERMINATE'."
)

In [ ]:
# Create the translator agent
translator_agent = ConversableAgent(
    name="Translator",
    llm_config={"config_list": config_list},
    system_message="You are a professional translator. Your task is to translate the given article into Spanish, maintaining the original meaning, tone, and structure. Once complete, include the translated article in your message and end with 'TERMINATE'."
)

In [ ]:
# Register the search function with the researcher agent
register_function(
    search_internet,
    caller=researcher_agent,
    executor=user_proxy_agent,
    name="search_internet",
    description="Search the internet using Tavily API"
)


In [ ]:
# Step 1: Initiate research chat
research_chat_result = user_proxy_agent.initiate_chat(
    researcher_agent,
    message="Please research the latest GPT-4.5 and provide key facts, figures, and credible sources."
)

# Extract research findings from the last message
research_findings = research_chat_result.chat_history[-1]["content"].replace("TERMINATE", "").strip()

# Step 2: Initiate writing chat with research findings
writing_chat_result = user_proxy_agent.initiate_chat(
    writer_agent,
    message=f"Based on the following research findings, please write a well-structured article:\n{research_findings}"
)

# Extract the article from the last message
article = writing_chat_result.chat_history[-1]["content"].replace("TERMINATE", "").strip()

# Step 3: Initiate translation chat with the article
translation_chat_result = user_proxy_agent.initiate_chat(
    translator_agent,
    message=f"Please translate the following article into Spanish:\n{article}"
)

# Extract the translated article from the last message
translated_article = translation_chat_result.chat_history[-1]["content"].replace("TERMINATE", "").strip()


In [ ]:
# Save results to files
print(research_findings)

print(article)

print(translated_article)

print("Results have been printed: research_findings, article, and translated_article_spanish")